## Case Study: Solve recaptcha v2 with 2captcha service and Selenium

Created by [tanyongsheng.net](https://tanyongsheng.net)

---

### Prerequisite
1. Buy 2Captcha credit at https://2captcha.com/?from=22013304 (Note: this is an affiliate link)

### Reference 
1. How to Solve Captcha / ReCaptcha - Python and 2captcha https://www.youtube.com/watch?v=R6QddZzCOwM & https://github.com/eupendra/2captcha_demo/blob/main/demo_requests.py
2. Bypass captcha in Python: https://2captcha.com/lang/python

(Credit: this is the code from 2captcha support team)

----

In [1]:
%pip install selenium
%pip install webdriver-manager
%pip install 2captcha-python
%pip install python-dotenv
# to inspect what python package imported in this jupyter notebook
%pip install watermark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
import os


In [3]:
_ = load_dotenv()

api_key = os.getenv("CAPTCHA_API_KEY")
investingnote_username = os.getenv("INVESTINGNOTE_USERNAME")
investingnote_password = os.getenv("INVESTINGNOTE_PASSWORD")
login_page_url = 'https://www.investingnote.com/users/sign_in'

In [4]:
# proxy1 = "xxxx:xxxx@xx.xxx.xx.xxx:xxxx"  # YOUR PROXY
driver = webdriver.Chrome()  # You can use another WebDriver (e.g., Firefox, Edge) if needed

driver.get(login_page_url)
time.sleep(5)

In [5]:
# fnd site key of InvestingNote website (or you could manually find )
src = driver.find_element(By.TAG_NAME, "iframe").get_attribute("src")
pattern = r'k=([^&]+)'
result = re.search(pattern, src)
sitekey = result.group(1)
print("sitekey: ", sitekey)

driver.find_element(By.ID, "user_login").send_keys(investingnote_username)
time.sleep(1)

driver.find_element(By.ID, "user_password").send_keys(investingnote_password)

sitekey:  6LfAHhcTAAAAAKIc0n_1WmG3P6DA4YIojZmH7wKq


In [6]:
# REference: https://2captcha.com/p/recaptcha_v2

try:
    ## Send sitekey and pageurl to 2Captcha API.
    data = {"key": api_key,
            "method": "userrecaptcha",
            "googlekey": sitekey,
            "json": 1,
            "pageurl": login_page_url,
            # "proxy": proxy1,
            "proxytype": "HTTP",
            }
    response = requests.post(f"https://2captcha.com/in.php?", data=data)
    print("Request has been sent: ", response.text)
    s = response.json()["request"]
    time.sleep(20)

    while True:
        solu = requests.get(f"https://2captcha.com/res.php?key={api_key}&action=get&json=1&id={s}").json()
        if solu["request"] == "CAPCHA_NOT_READY":
            print(solu["request"])
            time.sleep(10)
        elif "ERROR" in solu["request"]:
            print(solu["request"])
            driver.close()
            driver.quit()
            exit(0)
        else:
            break
            
    for key, value in solu.items():
        print(key, ": ", value)

    solu = solu["request"]
    time.sleep(2)

    driver.execute_script("document.getElementsByName('g-recaptcha-response')[0].value = arguments[0];", solu)
    iframe = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[2]/div/div/div/form/fieldset/section[3]/div/div/div/iframe")
    driver.switch_to.frame(iframe)
    el = driver.find_element(By.ID, "recaptcha-token")
    driver.execute_script(f"arguments[0].setAttribute('value', '{solu}');", el)
    time.sleep(1)
    driver.switch_to.default_content()

    script = """document.getElementById('sign-in-btn').removeAttribute('disabled');"""  # Making the button visible
    driver.execute_script(script)

    time.sleep(2)
    print("Answer inserted")
    time.sleep(2)
    driver.find_element(By.ID, "sign-in-btn").click()  # Click on the authorization button
    time.sleep(20)

except Exception as ex:
    print(ex)

finally:
    time.sleep(1)
    driver.close()
    driver.quit()


Request has been sent:  {"status":1,"request":"75955022846"}
CAPCHA_NOT_READY
CAPCHA_NOT_READY
status :  1
request :  03AFcWeA4T4UQERAQ48eKBkRZmWLGRxpzYUXvpnCuhoEREYfHSfJryxOfHsVMTctDP29ImTpVbzr8qES_z1UfkIZA3mDlJcn3V5YUFZ7w40rgruI4Pn-XCtL8t5WdF1PXTCGeiBT0LgQgBnHp1_qEFXbIk0ICBu6bAxXE3gb-QiXO7j0NwTAJlRShntntVxrGkyNa1CvS163JEqhdnJ_GxzivQJPgcNXZdDVrGpD8EZnYlva4xnMoVRnRKhPfEQ3mwYUPaNacEJnTcZ_phnclIBnoXISIsSbWMSW3QK8m95W3bK7KWhB0fo2PQP6vtmQyog1xfAjUYH0HFVyESNV5mczxKbb71_Kg-d5QTi_GTMj6F61CNELSOB78i3btTCpHVcj77JNATYQXSJZRvVkXuE6a4nVm_qhlFEZbrxkwIpLu-pu9YZYnaIt6_8t6frUXzeDM_8Mwe0mE3Ts1iQ3BP_Jo9NN-bpzM-aAw2wJ1C9ONPhYo165cE9N62NFqXX8WjMUdhbOMFMqoXQnnNnwAgSzTK9irDPS7SJ8lpo8mVz1Ol5L7RZpF3geBjsfuxRzceYjk5R5StLIn4GmdrvdJIeqjbDdtwhXqZB-6QBEFOxSg7RS57kvg3UBFMNZ73IG_35IpHESlKsmw4cqhROzehOwu3g-mYZe70mOEK3aFlKP9Z71FqFHhkvl8Vch7rTTGmlSGps8VO8zNx3gkz06JTdKnDUP6cfdURIQ
Answer inserted


## Computing Environment

In [7]:
%load_ext watermark

%watermark
%watermark  --iversions
%watermark -u -n -t -z

Last updated: 2024-03-17T06:51:30.854360+00:00

Python implementation: CPython
Python version       : 3.10.13
IPython version      : 8.22.2

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.75-060175-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit

re      : 2.2.1
requests: 2.31.0
selenium: 4.18.1

Last updated: Sun Mar 17 2024 06:51:30UTC

